In [ ]:
!pip3 install torch torchvision torchaudio
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
from torchvision import transforms
import os
from pytorch_i3d import InceptionI3d
import moviepy.editor as mp

!pip3 install av
import av
print(av.__version__)

# !pip3 install opencv-python
!pip3 install opencv-contrib-python
import cv2

In [ ]:
!nvidia-smi
print(torch.cuda.is_available())

In [ ]:
def video_to_rgb(video_path, video_name, new_rgb_root = "./test_rgb"):
    os.makedirs(os.path.join(new_rgb_root, video_name), exist_ok=True)
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 1
    while success:
        cv2.imwrite(os.path.join(new_rgb_root, video_name, video_name + "-%s.jpg" % str(count).zfill(6)), image)
        success, image = vidcap.read()
        count += 1

In [ ]:
import os
dir = "test_videos"
for category in os.listdir(dir):
  for video in os.listdir(os.path.join(dir, category)):
    video_to_rgb(os.path.join("test_videos", category, video), video.split(".")[0])

# TESTING

In [ ]:
# print(torch.cuda.device(0))
# print(torch.cuda.get_device_name(0))
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print(device)
rgb_i3d = InceptionI3d(400, in_channels=3)
rgb_i3d.replace_logits(8)
rgb_i3d.load_state_dict(torch.load("./newmodels_info/Training3/bnModel005000.pt", map_location=torch.device(device)))
rgb_i3d.eval()
flow_i3d = InceptionI3d(400, in_channels=2)
flow_i3d.replace_logits(8)
flow_i3d.load_state_dict(torch.load("./newmodels_info/Training1/flow_anamoly005000.pt", map_location=torch.device(device)))
flow_i3d.eval()

In [ ]:
from charades_dataset import load_rgb_frames, load_flow_frames, video_to_tensor
import videotransforms

cropTransform = transforms.Compose([videotransforms.CenterCrop(224)])

def predict(rgb_root, flow_root, video):
  rgb_imgs = load_rgb_frames(rgb_root, video, 1, int(len(os.listdir(os.path.join(rgb_root, video)))))
  rgb_imgs = cropTransform(rgb_imgs)
  rgb_input = video_to_tensor(rgb_imgs)
  rgb_input = rgb_input[None] # batch size of 1

  flow_imgs = load_flow_frames(flow_root, video, 1, int(len(os.listdir(os.path.join(flow_root, video)))/2))
  flow_imgs = cropTransform(flow_imgs)
  flow_input = video_to_tensor(flow_imgs)
  flow_input = flow_input[None] # batch size of 1
  with torch.no_grad():
      rgb_prediction = rgb_i3d(rgb_input)
      flow_prediction = flow_i3d(flow_input)
  prediction = (rgb_prediction + flow_prediction)/2.0
  averaged_prediction = torch.mean(prediction, dim=2)[0]
  normalized_prediction = torch.softmax(averaged_prediction, dim=0)
  print("PREDICTION|"+str(video)+"|")
  for p in normalized_prediction:
    print(p.item(), end=" ")
  print("")

In [ ]:
import os

for category in os.listdir("test_videos"):
  for video in os.listdir(os.path.join("test_videos", category)):
    predict("test_rgb", "test_flow", video.split(".")[0])

# Analysis